## Webscrapping do dólar pela B3
    Matheus Laidler

Não conheço nenhuma api da B3 em que eu possa buscar a cotação do dólar em tempo real. No entanto, podemos buscar essa informação direto pelo site deles. Essa informação é atualizada durante o dia, de forma que precisamos verificar a data de atualização da cotação.

In [7]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import time
import multiprocessing as mp
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import poisson
from scipy.optimize import curve_fit

In [8]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

def bot_b3_dolar_fetch():
    driver = webdriver.Firefox( service=FirefoxService(GeckoDriverManager().install())
)#nosso bot que irá abrir o navegador e procurar os sites
    url = "https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/consultas/clearing-de-cambio/indicadores/taxas-de-cambio-referencial/"
    driver.get(url)
    time.sleep(3) #Python é mais rapido que o carregamento dos dados
    botao_aceitar_cookies = driver.find_element("xpath", '''/html/body/div[4]/div[3]/div/div[1]/div/div[2]/div/button[3]''')
    ##clicar
    driver.execute_script("arguments[0].click()", botao_aceitar_cookies)
    # Precisa mudar o foco pro iframe de onde está a cotação:
    # Aguarda até que o iframe esteja presente e muda o contexto para ele
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "bvmf_iframe"))
    )
    driver.switch_to.frame(iframe)

    # Agora, dentro do contexto do iframe, localize o elemento desejado
    cot_dolar = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//h4[@class='mb-0']"))
    ).text

    cot_dolar = float(cot_dolar.split(' ')[0].replace(',', '.'))

    data_att = driver.find_element("xpath", '''
    /html/body/app-root/app-rates/div/form/div/div/div/div/div/div[2]/div[3]/p
    ''').get_attribute('outerHTML').split(' ')[-1].split('<')[0]

    return cot_dolar, data_att

In [9]:
cot_dolar, dia = bot_b3_dolar_fetch()

In [10]:
cot_dolar

5.6517

In [11]:
dia

'10/09/2024'